In [18]:
def probAttr(data,attr,val):
    Total=data.shape[0]
    from collections import Counter
    cnt=Counter(x for x in data[attr])
    return cnt[val],cnt[val]/Total

In [19]:
def probAttrConcept(data,attr,val,concept,cVal,countConcept):
    count={}
    prob={}
    C = data[concept]
    A = data[attr]
    for a in range(len(data[attr])):
        for v in val:
            if(A[a]==v and C[a]==cVal):
                if v not in count:
                    count[v]=1
                else:
                    count[v]=count[v]+1
    for a in count:
        prob[a] = count[a]/countConcept
    return prob

In [20]:
def train(data,AttributeList,concept):
    Attr={}
    probability_list={}
    #Get attribute values
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
    
    conceptVals = list(set(data[concept]))
    conceptProbs = {}
    countConcept={}
    AttrConcept = {}
    for cVal in conceptVals:
        countConcept[cVal],conceptProbs[cVal] = probAttr(data,concept,cVal)
    
    for val in Attr:
        probability_list[val]={}
        AttrConcept[val] = {}
        for v in Attr[val]:
            a,probability_list[val][v]=probAttr(data,val,v)
        for cVal in conceptVals:
            AttrConcept[val][cVal]=probAttrConcept(data,val,Attr[val],concept,cVal,countConcept[cVal])
        
    print("P(A) : ",conceptProbs,"\n")
    print("P(X/A) : ",AttrConcept,"\n")
    print("P(X) : ",probability_list,"\n")
    return conceptProbs,AttrConcept,probability_list

In [26]:
def test(examples,AttributeList,conceptProbs,AttrConcept,probability_list,data,concept_list,Total):
    misclassification_count=0
    Total1 = len(examples)-1
    for ex in range(1,len(examples)):
        px={}
        for c in concept_list:
            for x in range(1,len(examples[ex])-1):
                for a in AttributeList:
                    if examples[ex][x] in AttrConcept[a][c]:  
                        if c not in px:
                            px[c] = 1*AttrConcept[a][c][examples[ex][x]]/probability_list[a][examples[ex][x]]
                        else:
                            px[c] = px[c]*AttrConcept[a][c][examples[ex][x]]/probability_list[a][examples[ex][x]]
            px[c] = px[c]*conceptProbs[c]
        print(px)
        classification = max(px,key=px.get)
        print("Classification :",classification,"Expected :",examples[ex][-1])
        if(classification!=examples[ex][-1]):
            misclassification_count+=1
    misclassification_rate=misclassification_count*100/Total1
    accuracy=100-misclassification_rate
    print("Misclassification Count={}".format(misclassification_count))
    print("Misclassification Rate={}%".format(misclassification_rate))
    print("Accuracy={}%".format(accuracy))

In [27]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('data_train1.csv')
    AttributeList=list(data)[:-1]
    num_of_attributes=len(AttributeList)
    concept=str(list(data)[-1])
    Total=data.shape[0]
    conceptProbs,AttrConcept,probability_list = train(data,AttributeList,concept)

    import csv
    with open('data_test1.csv')  as csvFile:
        examples = [list(line) for line in csv.reader(csvFile)]
    concept_list =list(Counter(x for x in data[concept]))
    test(examples,AttributeList,conceptProbs,AttrConcept,probability_list,data,concept_list,Total)

In [28]:
main()

P(A) :  {'Yes': 0.6428571428571429, 'No': 0.35714285714285715} 

P(X/A) :  {'Outlook': {'Yes': {'Overcast': 0.4444444444444444, 'Rain': 0.3333333333333333, 'Sunny': 0.2222222222222222}, 'No': {'Sunny': 0.6, 'Rain': 0.4}}, 'Temperature': {'Yes': {'Hot': 0.2222222222222222, 'Mild': 0.4444444444444444, 'Cool': 0.3333333333333333}, 'No': {'Hot': 0.4, 'Cool': 0.2, 'Mild': 0.4}}, 'Humidity': {'Yes': {'High': 0.3333333333333333, 'Normal': 0.6666666666666666}, 'No': {'High': 0.8, 'Normal': 0.2}}, 'Wind': {'Yes': {'Weak': 0.6666666666666666, 'Strong': 0.3333333333333333}, 'No': {'Weak': 0.4, 'Strong': 0.6}}} 

P(X) :  {'Outlook': {'Rain': 0.35714285714285715, 'Sunny': 0.35714285714285715, 'Overcast': 0.2857142857142857}, 'Temperature': {'Mild': 0.42857142857142855, 'Cool': 0.2857142857142857, 'Hot': 0.2857142857142857}, 'Humidity': {'Normal': 0.5, 'High': 0.5}, 'Wind': {'Strong': 0.42857142857142855, 'Weak': 0.5714285714285714}} 

{'No': 0.9408000000000002, 'Yes': 0.2419753086419753}
Classifica

/home/sahyadri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """
